In [3]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import tensorflow as tf
import numpy as np
from keras import layers
import numpy as np

In [ ]:
df = pd.read_json('./News_Category_Dataset_v3.json', encoding='utf-8', lines=True)

In [ ]:
# I decided to merge similar categories to improve accuracy
df.category = df.category.map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)
df.category = df.category.map(lambda x: "ARTS & CULTURE" if x == "ARTS" else x)
df.category = df.category.map(lambda x: "ARTS & CULTURE" if x == "CULTURE & ARTS" else x)
df.category = df.category.map(lambda x: "STYLE & BEAUTY" if x == "STYLE" else x)

In [ ]:
# trying to improve accuracy by combining headline and short description
df['text'] = df.headline + " " + df.short_description

In [ ]:
X = df['text']
y = df['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)